In [72]:
# Importing the libraries
import tensorflow as tf
import os
import shutil
import numpy as np
import cv2
import h5py
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import *

In [73]:
# Part 1 - Data Preprocessing

train_dir = './train2'
validation_dir = './validation2'
test_dir = './test2'

# training set
train_dogs_dir = os.path.join(train_dir, 'dogs')
train_cats_dir = os.path.join(train_dir, 'cats')

# validation set
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Preprocessing the Training set
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range = 40,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True)
training_set = train_datagen.flow_from_directory(
        train_dir,
        target_size=(64, 64),  # make training much faster
        batch_size=32,  # how many images in each batch, 32 is a classic default
        class_mode='binary')  # cat, dog: binary

# Preprocessing the Test set
val_datagen = ImageDataGenerator(rescale=1./255)
val_set = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [74]:
# Part 2 - Building the CNN

# Initialising the CNN
cnn = tf.keras.models.Sequential()  # an object as sequence of layers

# Step 1 - Convolution, use add method to add layer
cnn.add(tf.keras.layers.Conv2D(filters=32,
                               kernel_size=3,
                               activation='relu',
                               input_shape=[64, 64, 3]))

# Step 2 - Pooling (Max Pooling)
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=64,
                               kernel_size=3,
                               activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a third convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=64,
                               kernel_size=3,
                               activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# # Step 4 - Fully Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Add dropout

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [75]:
%%time
# Part 3 - Training the CNN

# Compiling the CNN
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the CNN on the Training set and evaluating it on the Val set
cnn.fit(x=training_set, validation_data=val_set, epochs=20, verbose=2)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 625 steps, validate for 157 steps
Epoch 1/20
625/625 - 130s - loss: 0.6844 - accuracy: 0.5552 - val_loss: 0.6449 - val_accuracy: 0.6142
Epoch 2/20
625/625 - 118s - loss: 0.6284 - accuracy: 0.6465 - val_loss: 0.6033 - val_accuracy: 0.6666
Epoch 3/20
625/625 - 128s - loss: 0.5888 - accuracy: 0.6892 - val_loss: 0.5433 - val_accuracy: 0.7270
Epoch 4/20
625/625 - 124s - loss: 0.5708 - accuracy: 0.7032 - val_loss: 0.5821 - val_accuracy: 0.6812
Epoch 5/20
625/625 - 120s - loss: 0.5440 - accuracy: 0.7242 - val_loss: 0.4792 - val_accuracy: 0.7716
Epoch 6/20
625/625 - 114s - loss: 0.5247 - accuracy: 0.7370 - val_loss: 0.4690 - val_accuracy: 0.7756
Epoch 7/20
625/625 - 102s - loss: 0.5072 - accuracy: 0.7511 - val_loss: 0.4920 - val_accuracy: 0.7516
Epoch 8/20
625/625 - 107s - loss: 0.4943 - accuracy: 0.7564 - val_loss: 0.4159 - val_accuracy: 0.8060
Epoch 9/20
625/625 - 109s - loss: 0.4816 - accuracy: 0.7677 - val_loss: 0.4744 - val_accur

In [76]:
# Part 4 - Making prediction

test_dir = './test2'

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=32,
    # multi-class: one-hot; 2-class: binary
    class_mode=None, 
    shuffle=False
)

Found 12500 images belonging to 1 classes.


In [77]:
%%time
y_cnn_pred = cnn.predict(test_generator)

Wall time: 44.8 s


In [78]:
y_cnn_pred[:10]

array([[0.8375372 ],
       [0.00786807],
       [0.2286997 ],
       [0.9953252 ],
       [0.8612121 ],
       [0.36466756],
       [0.06960865],
       [0.9764771 ],
       [0.9657177 ],
       [0.07868503]], dtype=float32)

In [79]:
print(type(y_cnn_pred))
y_cnn_pred = np.clip(y_cnn_pred, a_min=0.005, a_max=0.995)

<class 'numpy.ndarray'>


In [96]:
y_cnn_pred[:10]

array([[0.8375372 ],
       [0.00786807],
       [0.2286997 ],
       [0.995     ],
       [0.8612121 ],
       [0.36466756],
       [0.06960865],
       [0.9764771 ],
       [0.9657177 ],
       [0.07868503]], dtype=float32)

In [103]:
y_pred_2 = [0.995 for x in y_cnn_pred if x > 0.5 else 0.005]

SyntaxError: invalid syntax (<ipython-input-103-f04a9c459134>, line 1)

In [102]:
len(y_pred_2)

6762

In [81]:
df = pd.read_csv('sample_submission.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12500 entries, 0 to 12499
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      12500 non-null  int64  
 1   label   12500 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 195.4 KB


In [82]:
test_filenames = test_generator.filenames

In [101]:
# Part 5: Submit

for i, f_name in enumerate(test_generator.filenames):
    index = int(f_name[f_name.rfind('\\')+1:f_name.rfind('.')])
    df.at[index - 1, 'label'] = y_pred_2[i]

IndexError: list index out of range

In [84]:
df.head(15)

,id,label
0,1,0.837537
1,2,0.981501
2,3,0.961401
3,4,0.561453
4,5,0.021561
5,6,0.270759
6,7,0.005000
7,8,0.328604
8,9,0.516487
9,10,0.007868


In [85]:
df.to_csv('submission_cnn5.csv', index=None)

In [86]:
# Save model
cnn.save('cnn_model2.h5')
cnn.save_weights('cnn_model2.weight')
print("CNN model saved!!!")

CNN model saved!!!


## Transfer Learning

In [87]:
import os
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model

In [88]:
# tf.keras.applications
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.resnet import ResNet101

In [90]:
pre_trained_model = ResNet101(input_shape = (64, 64, 3), # input_shape, cannot smaller than 32, 32
                             include_top = False, # whether to include the fully-connected layer at the top of the network.
                             weights='imagenet')

In [91]:
for layer in pre_trained_model.layers:
    layer.trainable = False

In [92]:
# prepare for fully connected
x = layers.Flatten()(pre_trained_model.output)
# we need to train fully connected layer
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
# output layer
x = layers.Dense(1, activation='sigmoid')(x)           

# Model
model = Model(pre_trained_model.input, x) 

opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, 
              loss='binary_crossentropy', 
              metrics=['acc'])

In [93]:
%%time
# Preprocessing the Training set
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range = 40,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True)
training_set = train_datagen.flow_from_directory(
        train_dir,
        target_size=(64, 64),  # make training much faster
        batch_size=32,  # how many images in each batch, 32 is a classic default
        class_mode='binary')  # cat, dog: binary

# Preprocessing the Test set
val_datagen = ImageDataGenerator(rescale=1./255)
val_set = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Wall time: 2.31 s


In [94]:
%%time
# Training the CNN on the Training set and evaluating it on the Val set
model.fit(x=training_set, validation_data=val_set, epochs=20, verbose=2)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 625 steps, validate for 157 steps
Epoch 1/20
625/625 - 226s - loss: 0.5892 - acc: 0.7156 - val_loss: 1.5126 - val_acc: 0.5000
Epoch 2/20
625/625 - 214s - loss: 0.5094 - acc: 0.7455 - val_loss: 1.4208 - val_acc: 0.5000
Epoch 3/20
625/625 - 213s - loss: 0.4956 - acc: 0.7577 - val_loss: 1.4475 - val_acc: 0.5000
Epoch 4/20
625/625 - 214s - loss: 0.4901 - acc: 0.7583 - val_loss: 1.2953 - val_acc: 0.5000
Epoch 5/20
625/625 - 214s - loss: 0.4833 - acc: 0.7646 - val_loss: 1.1210 - val_acc: 0.5000
Epoch 6/20


KeyboardInterrupt: 

In [27]:
os.listdir(test_dir)

['test']

In [28]:
imgs = []
ids = []
labels = []
for name in test_filenames:
    ids.append(name.split('.')[0])
#     img_path = os.path.join(test_dir, name)
#     img = image.load_img(img_path, target_size=(150, 150))
#     x = image.img_to_array(img)
#     x = np.expand_dims(x, axis=0)
#     x = preprocess_input(x)
#     label = model.predict(x)
#     if label[0] > 0.5:
#         labels.append(0.995)
#     else:
#         labels.append(0.005)
#     imgs.append(x)

In [61]:
df = pd.read_csv('sample_submission.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12500 entries, 0 to 12499
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      12500 non-null  int64  
 1   label   12500 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 195.4 KB


In [62]:
%%time
y_rn50_pred = model.predict(test_generator)

Wall time: 1min 9s


In [63]:
y_rn50_pred = np.clip(y_rn50_pred, a_min=0.005, a_max=0.995)

In [64]:
y_rn50_pred[:10]

array([[0.03318811],
       [0.0343309 ],
       [0.03293698],
       [0.03680131],
       [0.03200804],
       [0.01541337],
       [0.03313997],
       [0.0129495 ],
       [0.01981016],
       [0.02595064]], dtype=float32)

In [65]:
# Part 5: Submit
for i, f_name in enumerate(test_generator.filenames):
    index = int(f_name[f_name.rfind('\\')+1:f_name.rfind('.')])
    df.at[index - 1, 'label'] = y_rn50_pred[i]

In [66]:
df.head(15)

,id,label
0,1,0.033188
1,2,0.022592
2,3,0.048237
3,4,0.026121
4,5,0.022366
5,6,0.020964
6,7,0.029738
7,8,0.028587
8,9,0.008980
9,10,0.034331


In [71]:
df.to_csv('submission_rn50.csv', index=None)

In [70]:
# Save model
model.save('resnet50_model.h5')
model.save_weights('resnet50_model.weight')
print("Transfer learning model saved!!!")

Transfer learning model saved!!!
